In [1]:
library(tidyverse)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [63]:
df_eff <- read_csv('effervescence.csv', col_types = 'fffnn')

In [64]:
df_eff %>% head()

Brand,Temp,Stirred,Order,Time
<fct>,<fct>,<fct>,<dbl>,<dbl>
name,6,yes,8,77.21547
name,23,yes,3,75.37855
name,40,yes,7,68.08492
store,6,yes,1,77.87371
store,23,yes,2,66.38436
store,40,yes,18,59.82388


In [146]:
lm_eff <- lm(Time ~ Brand * Temp, data = df_eff)
aov_eff <- aov(lm_eff)
anova_eff <- anova(lm_eff)

In [105]:
me_table <- as_tibble(anova_eff[,1:3])
me_table <- cbind('Source' = c('Brand', 'Temp', 'Brand*Temp', 'Residual'), 
                  me_table, 
                  'Error Term' = c("MSA", "MSB", "MSAB", "MSE"))
me_table

Source,Df,Sum Sq,Mean Sq,Error Term
<chr>,<int>,<dbl>,<dbl>,<chr>
Brand,1,342.0072,342.007154,MSA
Temp,2,1654.7366,827.368276,MSB
Brand*Temp,2,231.8519,115.925956,MSAB
Residual,42,141.1685,3.361154,MSE


In [164]:
me_table <- as_tibble(summary(aov_eff)[[1]][,1:3])
me_table <- cbind('Source' = c('Brand', 'Temp', 'Brand*Temp', 'Residual'), 
                  me_table, 
                  'Error Term' = c("MSA", "MSB", "MSAB", "MSE"))

MSA <-  me_table[1, 4]
MSB <-  me_table[2, 4]
MSAB <- me_table[3, 4]
MSE <-  me_table[4, 4]

a <- length(levels(df_eff$Brand))
b <- length(levels(df_eff$Temp))
n <- nrow(df_eff)/(a*b)

sigma_ab <- (MSAB - MSE) / n
sigma_a <- (MSA - MSAB) / (b * n)
sigma_b <- (MSB - MSAB) / (a*n)

f_scores <- c(MSA/MSAB,MSB/MSAB,MSAB/MSE, NA)

error_dof <- c(rep(tail(me_table$Df, n = 2)[1], 2), tail(me_table$Df, n = 1)[1], NA)

me_table['Error Df'] <- error_dof

me_table["F Score"] <- f_scores
f_test <- round(1 - pf(me_table['F Score'][[1]], 
                       me_table['Df'][[1]], 
                       me_table['Error Df'][[1]]),
                4)

me_table['Pr>F'] <- f_test



name_mean <- df_eff %>% filter(Brand == 'name') %>% select(Time) %>% unlist() %>% mean()
brand_fe <- df_eff %>% group_by(Brand) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)

name_mean <- df_eff %>% filter(Temp == '6') %>% select(Time) %>% unlist() %>% mean()
temp_fe <- df_eff %>% group_by(Temp) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)

knitr::kable(me_table, caption = 'Mixed Effects Models')

knitr::kable(brand_fe, caption = 'Model4: Brand is Fixed and Temperature is Random')
knitr::kable(cbind(c('Temp', 'Brand*Temp', 'Residual'), 
                   c(sigma_b, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(temp_fe, caption = 'Model5: Brand is Random and Temperature is Fixed')
knitr::kable(cbind(c('Brand', 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(cbind(c('Brand', 'Temp' , 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_b,sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'), 
             caption = 'Model6: Brand and Temperature are Random')



Table: Mixed Effects Models

|Source     | Df|    Sum Sq|    Mean Sq|Error Term | Error Df|   F Score|   Pr>F|
|:----------|--:|---------:|----------:|:----------|--------:|---------:|------:|
|Brand      |  1|  342.0072| 342.007154|MSA        |        2|  2.950221| 0.2280|
|Temp       |  2| 1654.7366| 827.368276|MSB        |        2|  7.137041| 0.1229|
|Brand*Temp |  2|  231.8519| 115.925956|MSAB       |       42| 34.489926| 0.0000|
|Residual   | 42|  141.1685|   3.361154|MSE        |       NA|        NA|     NA|



Table: Model4: Brand is Fixed and Temperature is Random

|Brand |     Mean|    Effect|
|:-----|--------:|---------:|
|name  | 73.44276|  0.000000|
|store | 68.10416| -5.338595|



|Cov Parm   |Estimate         |
|:----------|:----------------|
|Temp       |44.4651449693106 |
|Brand*Temp |14.0706002662374 |
|Residual   |3.36115408186724 |



Table: Model5: Brand is Random and Temperature is Fixed

|Temp |     Mean|     Effect|
|:----|--------:|----------:|
|6    | 78.00561|   0.000000|
|23   | 70.69044|  -7.315177|
|40   | 63.62433| -14.381286|



|Cov Parm   |Estimate         |
|:----------|:----------------|
|Brand      |9.42004992022257 |
|Brand*Temp |14.0706002662374 |
|Residual   |3.36115408186724 |



Table: Model6: Brand and Temperature are Random

|Cov Parm   |Estimate         |
|:----------|:----------------|
|Brand      |9.42004992022257 |
|Temp       |44.4651449693106 |
|Brand*Temp |14.0706002662374 |
|Residual   |3.36115408186724 |

In [161]:
library(lmerTest)
lm_mixed <- lmer(Time ~ (1|Brand) + Temp + (1|Brand:Temp), data = df_eff)
summary(lm_mixed)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Time ~ (1 | Brand) + Temp + (1 | Brand:Temp)
   Data: df_eff

REML criterion at convergence: 202.3

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.93345 -0.48381  0.00386  0.74725  1.67968 

Random effects:
 Groups     Name        Variance Std.Dev.
 Brand:Temp (Intercept) 14.071   3.751   
 Brand      (Intercept)  9.420   3.069   
 Residual                3.361   1.833   
Number of obs: 48, groups:  Brand:Temp, 6; Brand, 2

Fixed effects:
            Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)   78.006      3.458   2.289  22.560 0.000972 ***
Temp23        -7.315      3.807   2.000  -1.922 0.194593    
Temp40       -14.381      3.807   2.000  -3.778 0.063467 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
       (Intr) Temp23
Temp23 -0.550       
Temp40 -0.550  0.500